# The Finnis-Sinclair potential

Reading:
- Finnis and Sinclair 1984, [A simple empirical N-body potential for transition metals](https://doi.org/10.1080/01418618408244210)
- Daw and Baskes 1984, [Embedded-atom method: Derivation and application to impurities, surfaces, and other defects in metals](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.29.6443)

Content:
1. the math of a glue potential in general & what's special in the Finnis-Sinclair version
2. components of the Finnis-Sinclair model (pair energy, glue density, glue energy)
3. running a simulation and visualizing properties

In [ ]:
import Pkg

Pkg.activate(".")
Pkg.instantiate()

In [ ]:
using Molly
using Plots
using Test
using SingleCrystal

In [ ]:
make_atom(name,mass) = Atom(name=name,mass=mass)
SingleCrystal.make_bcc_unitcell("W", 2.3, make_atom)

## The math 💖

The potential energy is composed out of a pair and a "glue" contribution
$$
U = U_{\text{pair}} + U_{\text{glue}}
$$

The pair contribution
$$
U_{\text{pair}} = \sum_{i,j>i}V_{ij}(r_{ij})
$$

where $r_{ij} = \left\| \vec{R}_j - \vec{R}_i|  \right\|_2$.

The energy contribution of the glue is taken for each atom
$$
U_{\text{glue}} = \sum_i f(\rho_i)
$$

where $\rho_i = \sum_j \phi(r_{ij})$ is the glue at atom $i$.

Combining both
$$
\begin{split}
U &= U_{\text{pair}} + U_{\text{glue}} \\
  &= \sum_{i,j>i}V_{ij}(r_{ij}) + \sum_i f(\rho_i) \\
\end{split}
$$

To compute the force on atom i with this potential we apply
$$
\vec{F}_i = - \partial_i U
$$

where $\partial_i$ is the wiggling of $\vec{R}_i$.

Applying the chain rule we obtain
$$
\begin{split}
\vec{F}_i &= - \sum_j V^\prime_{ij}(r_{ij}) \left( -\frac{\vec{R}_{ij}}{r_{ij}} \right) + f^\prime_i(\rho_i) \sum_j \phi_{j}^\prime(r_{ij}) \left( -\frac{\vec{R}_{ij}}{r_{ij}} \right) + \sum_j f^\prime_j(\rho_j) \phi_i(r_{ji}) \left( \frac{\vec{R}_{ji}}{r_{ji}} \right) \\
    &= \sum_j \left[ V^\prime_{ij}(r_{ij}) + f^\prime_i(\rho_i)  \phi_{j}^\prime(r_{ij}) + f^\prime_j(\rho_j) \phi_i(r_{ji}) \right] \frac{\vec{R}_{ij}}{r_{ij}}
\end{split}
$$

The interesting part is the second equation, because this we can directly use to implement the force updates, looping the neighbouring pairs in parallel ($j$ is a neighbouring atom of atom $i$). 

## The model

Let's look into $f$, $\phi$ and $V$. 

### Parameterisations

Parameterisation by Finnis et al. 1984, _A simple empirical N-body potential for transition metals_

| element | d | A | $\beta$ | c | $c_0$ | $c_1$ | $c_2$ | 
| --- | --- | --- | --- | --- | --- | --- | --- |
| V  | 3.692767 | 2.010637 | 0   | 3.8  | -0.8816318 | 1.4907756   | -0.3976370 |
| Nb | 3.915354 | 3.013789 | 0   | 4.2  | -1.5640104 | 2.0055779   | -0.4663764 |
| Ta | 4.076980 | 2.591061 | 0   | 4.2  | 1.2157373  | 0.0271471   | -0.1217350 |
| Cr | 3.915720 | 1.453418 | 1.8 | 2.9  | 29.1429813 | -23.3975027 | 4.7578297 |
| Mo | 4.114825 | 1.887117 | 0   | 3.25 | 43.4475218 | -31.9332978 | 6.0804249 |
| W  | 4.400224 | 1.896373 | 0   | 3.25 | 47.1346499 | -33.7665655 | 6.2541999 |
| Fe | 3.699579 | 1.889846 | 1.8 | 3.4  | 1.2110601  | -0.7510840  | 0.1380773 |

Note that this potential assumes *metal* units, e.g. eV, K and so on: https://lammps.sandia.gov/doc/units.html

Collecting the parameters

In [ ]:
fs84, elements, masses, bcc_lattice_constants, reference_energies = Molly.get_finnissinclair1984(true);

### Glue contribution - $\phi(r)$

To compute the glue density $\rho$ for an atom, we need to compute the contributions from its neighbours (self-contribution is ignored). The neighbour contributions are obtained using $\phi(r)$:

$$
\phi(r) = (r-d)^2 + \beta (r-d)^3/d
$$

$d$ and $\beta$ are model parameters.

In [ ]:
r = collect(range(0, stop=6, length=1000));

ɸs = []
for el in elements
    d, β = fs84.singles[el].d, fs84.singles[el].β
    ɸ = Molly.glue.(r, β, d)
    append!(ɸs,[ɸ])
end

plot(r, ɸs, label=hcat(elements...), xlabel="r", ylabel="phi(r)", title="Glue contributions")

Interestingly we find that some $\phi$ do indeed take on negative values. This is interesting because of the choice for $f$ by Finnis and Sinclair.

### Glue energy - $f(\rho)$

Computing an energy based on local glue values

$$
u_\text{glue} = -A \cdot \sqrt{\rho}
$$

$$
\rho = \sum_{j \in \text{neighborhood}(i)} \phi(r_{ij})
$$

From a physical point of view it makes sense to expect $\rho > 0$, but in some situations we still might obtain  $\rho < 0$, and have a problem, unless we allow for complex energy values.

Let's plot $u_{\text{glue}}$ for $\rho \in \mathbb{R}^+$.

In [ ]:
ρ = collect(range(0, stop=50, length=100)) 

uₙs = []
for el in elements
    uₙ = Molly.Uglue.(ρ, fs84.singles[el].A)
    append!(uₙs,[uₙ])
end

plot(ρ, uₙs, label=hcat(elements...), xlabel="Glue density", ylabel="Glue energy", title="Glue energy varying with glue density")

### Pair energy - $V(r)$

The pair energy is defined as
$$
V_{ij}(r_{ij}) = 
\begin{cases} 
r \le c, & (r-c)^2 \left( c_0 + c_1 r + c_2 r^2 \right) \\
r > c, & 0 \\
\end{cases}
$$


Let's have a look

In [ ]:
Vs = []
element_pairs = []
for el in elements
    el_pair = string(el,el)
    c, c₀, c₁, c₂ = fs84.pairs[el_pair].c, fs84.pairs[el_pair].c₀, fs84.pairs[el_pair].c₁, fs84.pairs[el_pair].c₂
    V = Molly.Upair.(r, c, c₀, c₁, c₂)
    append!(Vs,[V])
    append!(element_pairs, [el_pair])
end

plot(r, Vs, label=hcat(element_pairs...), xlabel="r", ylabel="Pair energy contribution", title="Pair energy vs separation")

## Simulating

Now that we have looked at the individual components of the model, let's simulate Tungsten near room temperature for a $3\times3\times3$ supercell of a body centered cubic crystal.

Creating the crystal

In [ ]:
nx = 3
ny = 3
nz = 3
el = "W"

a = bcc_lattice_constants[el]
unitcell = SingleCrystal.make_bcc_unitcell(el, a, make_atom)
supercell = SingleCrystal.make_supercell(unitcell, nx=nx, ny=ny, nz=nz)
n_atoms = length(supercell.atoms)

Setting simulation specs

In [ ]:
T = 300. # Kelvin
T *= fs84.kb
n_steps = 5000
dt = .002 # ps; ns = 1e-9, ps = 1e-12, fs = 1e-15

general_inters = (fs84,)
velocities = [velocity(atom.mass, T, dims=3) for atom in supercell.atoms]
nb_matrix = trues(n_atoms,n_atoms)
dist_cutoff = 2. * a
nf = DistanceNeighbourFinder(nb_matrix, 1, dist_cutoff)
# thermostat = NoThermostat()
thermostat = AndersenThermostat(1.)

loggers = Dict(
    "temperature" => TemperatureLogger(1),
    "pot" => EnergyLogger(1),
    "coords" => CoordinateLogger(100),
    "velos" => VelocityLogger(1),
    "forces" => ForceLogger(100),
    "glue" => GlueDensityLogger(1),
#     "writer" => StructureWriter(5,"traj_bcc.pdb")
)

s = Simulation(
    simulator=VelocityVerlet(), 
    atoms=supercell.atoms, 
    general_inters=general_inters,
    coords=[SVector{3}(v) for v in supercell.positions], 
    velocities=velocities,
    temperature=T, 
    box_size=supercell.edge_lengths[1],
    timestep=dt,
    n_steps=n_steps,
    neighbour_finder=nf,
    loggers=loggers,
)

Running the simulation (this should take about 30s)

In [ ]:
simulate!(s)

Visualizing content of the loggers

In [ ]:
x = collect(1:length(s.loggers["glue"].glue_densities))
y = [[arr[i] for arr in s.loggers["glue"].glue_densities] for i in 1:n_atoms]
plot(x, y, title="glue densities", legend=false)

In [ ]:
x = collect(1:length(s.loggers["temperature"].temperatures))
y = s.loggers["temperature"].temperatures/fs84.kb
plot(x,y,title="Temperature",legend=false)

In [ ]:
x = collect(1:length(s.loggers["pot"].energies))
y = s.loggers["pot"].energies / length(s.coords)
plot(x,y,title="Potential energy per atom",legend=false)